In [1]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import transforms, datasets

from tqdm import tqdm
from dataset.cifar10 import load_dataset
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from sklearn.model_selection import train_test_split
from resnet_generator import Generator, Critic
import timm
import yaml
from munch import Munch
import logging
import PyramidNet as PYRN
import numpy as np

In [2]:
gpu_id = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %s:" %(gpu_id), use_cuda)

GPU device 2: True


In [3]:
with open('config/c10.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

In [4]:
checkpoint = './log/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 8

In [5]:
# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# train params
epochs = 375
batch_size = 150
init_lr = 0.1
weight_decay = 0.0001
labels = 10
momentum = 0.9

In [6]:
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                         std=[x/255.0 for x in [63.0, 62.1, 66.7]])    

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
    ])

In [7]:
train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=True, download=True, transform=transform_train),
        batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
numberofclass = 10

Files already downloaded and verified


In [8]:
model_g = Generator(args, img_size=224, max_conv_dim=256)
model_g.load_state_dict(torch.load('model_best.pth.tar')['state_dict'])
model_g = model_g.cuda()

In [9]:
model = PYRN.PyramidNet('cifar10', 272, 200, 10, True)
model = model.cuda()
# model = torch.nn.DataParallel(model)

In [10]:
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay, nesterov=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
step_lr = torch.optim.lr_scheduler.StepLR(optimizer, 150, gamma=0.1, last_epoch=-1)
loss_func = nn.CrossEntropyLoss().cuda()

In [11]:
# Resume
# if resume:
#     print('==> Resuming from checkpoint..')
#     checkpoint = os.path.dirname(resume)
# #     checkpoint = torch.load(resume)
#     resume = torch.load(resume)
#     best_acc = resume['best_acc']
#     start_epoch = resume['epoch']
#     model.load_state_dict(resume['state_dict'])
#     optimizer.load_state_dict(resume['optimizer'])
#     logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
# else:
logger = Logger(os.path.join(checkpoint, 'log.txt'))
logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train top-1 Acc', 'Train top-5 Acc', 'Valid top-1 Acc', 'Valid top-5 Acc'])

In [ ]:
for e in range(epochs):
    model.train()
    model_g.eval()
    train_loss, train_top1,train_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    bar = tqdm(total=len(train_loader), leave=False)
    for x, t in train_loader:
        x, t = x.cuda(), t.cuda()
        r = np.random.random(size=x.size(0))
        r_batch = x[r < 0.01]
        if len(r_batch) > 0:
            inputs = F.interpolate(r_batch, size=224, mode='bicubic')
            g_out = model_g(inputs)
            inputs = inputs + g_out
            inputs = F.interpolate(inputs, size=32, mode='bicubic')
            x[r < 0.01] = inputs
#             x = x.detach()
            
        y = model(x)
        loss = loss_func(y, t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        err1 = accuracy(y.data, t, topk=(1,))
        err5 = accuracy(y.data, t, topk=(5,))
        train_loss.update(float(loss.data), x.size(0))
        train_top1.update(float(err1[0]), x.size(0))
        train_top5.update(float(err5[0]), x.size(0))
        bar.set_description("Epoch: {:d} / {:d}, LR:{:.6f}, Loss: {:.6f}, Top1E: {:.2f}, Top5E: {:.2f}".format(e, epochs, 
            optimizer.param_groups[0]["lr"], train_loss.avg, train_top1.avg, train_top5.avg), refresh=True)
        bar.update()
    bar.close()
    scheduler.step()
    step_lr.step()
    model.eval()
    test_loss, test_top1, test_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    for x, t in tqdm(test_loader, total=len(test_loader), leave=False):
        with torch.no_grad():
            x, t = Variable(x.cuda()), Variable(t.cuda())
            y = model(x)
            loss = loss_func(y, t)
            err1 = accuracy(y.data, t, topk=(1,))
            err5 = accuracy(y.data, t, topk=(5,))
            test_loss.update(float(loss.data), x.size(0))
            test_top1.update(float(err1[0]), x.size(0))
            test_top5.update(float(err5[0]), x.size(0))

    if (e + 1) % 100 == 0:
        torch.save({
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }, os.path.join(checkpoint, "{}.tar".format(e + 1)))

    lr = optimizer.param_groups[0]["lr"]
    logger.append([lr, train_loss.avg, test_loss.avg, train_top1.avg, train_top5.avg, test_top1.avg, test_top5.avg])

  0%|          | 0/334 [00:00<?, ?it/s]/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Epoch: 242 / 375, LR:0.002796, Loss: 0.000457, Top1E: 0.00, Top5E: 0.00:  64%|██████▍   | 214/334 [03:05<01:43,  1.16it/s]